# Submarine Sonar: Mine vs Rock Classification

## Can We Teach a Machine to Hear the Difference Between Death and a Pebble?

---

**Alright, let's dive in (pun absolutely intended).**

So here's the deal - sonar is basically underwater echolocation. Submarines send out sound waves, those waves bounce off stuff, and the pattern of echoes tells you what's out there. Simple concept, terrifying stakes.

**What we're working with:**
- **208 sonar returns** - yeah, it's small, but it's a classic
- **60 frequency bands** - each one measures how much energy bounced back at that frequency (0.0 to 1.0)
- **The signal:** A frequency-modulated chirp - starts low, goes high
- **Two targets:** Rocks (R) and Mines (M) - metal cylinders that go BOOM
- **Different angles:** 90 degrees coverage for mines, 180 for rocks

**The question:** Can we look at 60 numbers and figure out if the submarine should chill or panic?

Let's find out.

---

## Part 1: Setting Up Shop

First things first - import the tools, load the data, see what we're dealing with.

In [ ]:
# The usual suspects
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Dimensionality reduction
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder

# The model army
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

# Metrics
from sklearn.metrics import (classification_report, confusion_matrix, 
                            accuracy_score, precision_score, recall_score,
                            f1_score, roc_auc_score, roc_curve)

import warnings
warnings.filterwarnings('ignore')

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("All systems loaded. Ready to ping.")

**What just happened:** We loaded pandas for data handling, numpy for math, matplotlib/seaborn for visuals, scipy for statistics, and a whole army of sklearn models. Think of it as assembling the Avengers, but for data science.

---

In [ ]:
# Load the data
df = pd.read_csv('Sonar.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nThat's {df.shape[0]} sonar readings, each with {df.shape[1]-1} frequency measurements + 1 label")
print(f"\nColumns: {df.shape[1]-1} frequency bands (Freq_1 to Freq_60) + Label (M/R)")

**What this tells us:** 208 rows, 61 columns. Each row is one sonar ping bouncing off something. Each of the 60 frequency columns tells us how much energy came back at that specific frequency. The last column tells us what the object actually was - Mine or Rock.

208 samples is tiny by modern standards, but this dataset is from 1988 and it's still used today because it's genuinely challenging.

---

In [ ]:
# First look at the data
df.head(10)

**Reading these numbers:** Each cell is an energy value between 0 and 1. Higher = more energy bounced back at that frequency. The 'Label' column shows R (Rock) or M (Mine). 

Notice how the values vary a lot - some frequencies get strong returns (0.8+), others barely register (0.01). That variation is where the signal hides.

---

## Part 2: Basic EDA - Getting to Know Our Data

Before we throw algorithms at this, let's actually understand what we're working with.

In [ ]:
# Target distribution - how many mines vs rocks?
print("TARGET DISTRIBUTION")
print("="*40)
target_counts = df['Label'].value_counts()
print(f"\nMines (M):  {target_counts['M']} ({target_counts['M']/len(df)*100:.1f}%)")
print(f"Rocks (R):  {target_counts['R']} ({target_counts['R']/len(df)*100:.1f}%)")
print(f"\nTotal: {len(df)} samples")

# Visualize
fig, ax = plt.subplots(figsize=(8, 5))
colors = ['#e74c3c', '#3498db']
bars = ax.bar(['Mine (M)\nDANGER', 'Rock (R)\nSafe'], 
              [target_counts['M'], target_counts['R']], 
              color=colors, edgecolor='black', linewidth=2)
ax.set_ylabel('Count')
ax.set_title('What Are We Classifying?')
for bar, count in zip(bars, [target_counts['M'], target_counts['R']]):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
            str(count), ha='center', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

**What this means:** 111 mines, 97 rocks. Pretty balanced - about 53% mines, 47% rocks. This is good news because we don't need to worry about class imbalance messing with our models. 

If it was like 95% rocks and 5% mines, a lazy model could just predict "rock" every time and look accurate while being useless.

---

In [ ]:
# Basic statistics of frequency bands
print("FREQUENCY BAND STATISTICS")
print("="*40)
freq_cols = [col for col in df.columns if col != 'Label']
stats_df = df[freq_cols].describe().T
stats_df['range'] = stats_df['max'] - stats_df['min']
print(f"\nNumber of frequency bands: {len(freq_cols)}")
print(f"\nOverall value range: {df[freq_cols].min().min():.4f} to {df[freq_cols].max().max():.4f}")
print(f"\nSummary across all bands:")
print(stats_df[['mean', 'std', 'min', 'max', 'range']].describe().round(4))

**Breaking this down:** All values are between 0 and 1 (normalized energy readings). The mean values hover around 0.03-0.3 depending on the frequency band. Standard deviations tell us how much variation exists - more variation = potentially more useful for classification.

The 'range' column shows how spread out each feature is. Features with tiny ranges might not be very informative.

---

In [ ]:
# Missing values check
print("MISSING VALUES CHECK")
print("="*40)
missing = df.isnull().sum().sum()
print(f"\nTotal missing values: {missing}")
if missing == 0:
    print("Clean dataset - no missing data to handle.")

# Duplicates check
print(f"\nDuplicate rows: {df.duplicated().sum()}")

**Nice:** No missing values, no duplicates. This dataset is clean - rare in the wild, but this is a curated benchmark dataset so it makes sense.

---

In [ ]:
# The money plot: Average frequency profile for Mines vs Rocks
# This is THE key visualization - shows how the two classes differ across frequencies

mines = df[df['Label'] == 'M'][freq_cols].mean()
rocks = df[df['Label'] == 'R'][freq_cols].mean()

fig, ax = plt.subplots(figsize=(14, 6))

x = range(1, 61)
ax.plot(x, mines.values, 'r-', linewidth=2, label='Mine (Metal Cylinder)', marker='o', markersize=4)
ax.plot(x, rocks.values, 'b-', linewidth=2, label='Rock', marker='s', markersize=4)
ax.fill_between(x, mines.values, rocks.values, alpha=0.3, color='gray')

ax.set_xlabel('Frequency Band (1-60)')
ax.set_ylabel('Mean Energy Response')
ax.set_title('Average Sonar Signature: Mine vs Rock\n(The gap between lines = how distinguishable they are)')
ax.legend(loc='upper right')
ax.set_xlim(1, 60)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**This is the most important plot in the whole analysis.** It shows the average sonar "fingerprint" of mines vs rocks.

**What to look for:**
- Where the lines are far apart = frequencies that are good at distinguishing mine from rock
- Where the lines overlap = frequencies that don't help much
- The gray shaded area shows the "difference zone" - bigger gaps = easier classification

You can see mines and rocks have different energy patterns, especially in the middle frequency bands. Metal reflects sonar differently than stone - that's the physics we're exploiting.

---

In [ ]:
# Distribution of energy across all frequency bands
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Overall distribution
all_values = df[freq_cols].values.flatten()
axes[0, 0].hist(all_values, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Energy Value')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Distribution of All Energy Readings')
axes[0, 0].axvline(x=np.mean(all_values), color='red', linestyle='--', label=f'Mean: {np.mean(all_values):.3f}')
axes[0, 0].legend()

# Mean energy per sample by class
df['mean_energy'] = df[freq_cols].mean(axis=1)
mines_energy = df[df['Label'] == 'M']['mean_energy']
rocks_energy = df[df['Label'] == 'R']['mean_energy']

axes[0, 1].hist(mines_energy, bins=20, alpha=0.6, color='red', label='Mines', edgecolor='black')
axes[0, 1].hist(rocks_energy, bins=20, alpha=0.6, color='blue', label='Rocks', edgecolor='black')
axes[0, 1].set_xlabel('Mean Energy per Sample')
axes[0, 1].set_ylabel('Count')
axes[0, 1].set_title('Mean Energy Distribution by Class')
axes[0, 1].legend()

# Variance per sample by class
df['var_energy'] = df[freq_cols].var(axis=1)
mines_var = df[df['Label'] == 'M']['var_energy']
rocks_var = df[df['Label'] == 'R']['var_energy']

axes[1, 0].hist(mines_var, bins=20, alpha=0.6, color='red', label='Mines', edgecolor='black')
axes[1, 0].hist(rocks_var, bins=20, alpha=0.6, color='blue', label='Rocks', edgecolor='black')
axes[1, 0].set_xlabel('Energy Variance per Sample')
axes[1, 0].set_ylabel('Count')
axes[1, 0].set_title('Energy Variance Distribution by Class')
axes[1, 0].legend()

# Box plot of selected frequencies
selected_freqs = ['Freq_10', 'Freq_20', 'Freq_30', 'Freq_40', 'Freq_50']
df_melt = df[selected_freqs + ['Label']].melt(id_vars='Label', var_name='Frequency', value_name='Energy')
sns.boxplot(data=df_melt, x='Frequency', y='Energy', hue='Label', palette=['red', 'blue'], ax=axes[1, 1])
axes[1, 1].set_title('Energy Distribution at Selected Frequencies')
axes[1, 1].legend(title='Class')

plt.tight_layout()
plt.show()

# Clean up temp columns
df.drop(['mean_energy', 'var_energy'], axis=1, inplace=True)

**Four plots, four insights:**

1. **Top-left (Overall distribution):** Most energy readings are low (0.0-0.2), with a right-skewed distribution. High energy returns are less common.

2. **Top-right (Mean energy by class):** Mines and rocks have overlapping mean energy distributions - you can't just say "high energy = mine". It's more nuanced than that.

3. **Bottom-left (Variance by class):** Similar story - variance alone doesn't cleanly separate the classes. We need the full 60-dimensional pattern.

4. **Bottom-right (Box plots):** At different frequencies, the separation varies. Some frequencies (like Freq_10, Freq_20) show clear differences between classes, others don't.

**Takeaway:** This isn't a simple problem. The classes are entangled, and we'll need sophisticated models that can learn complex patterns.

---

In [ ]:
# Correlation heatmap - how do frequency bands relate to each other?
fig, ax = plt.subplots(figsize=(14, 12))

corr_matrix = df[freq_cols].corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))  # Upper triangle mask

sns.heatmap(corr_matrix, mask=mask, cmap='RdBu_r', center=0, 
            square=True, linewidths=0.5, ax=ax,
            cbar_kws={'label': 'Correlation', 'shrink': 0.8})
ax.set_title('Correlation Between Frequency Bands\n(Red = positive correlation, Blue = negative)')

plt.tight_layout()
plt.show()

# Find highly correlated pairs
high_corr = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.9:
            high_corr.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))

print(f"\nHighly correlated pairs (|r| > 0.9): {len(high_corr)}")
if len(high_corr) > 0:
    print("\nTop 5:")
    for f1, f2, r in sorted(high_corr, key=lambda x: abs(x[2]), reverse=True)[:5]:
        print(f"  {f1} <-> {f2}: {r:.3f}")

**What the correlation heatmap reveals:**

- **Adjacent frequencies are correlated** (the diagonal red band) - makes physical sense, neighboring frequency bands behave similarly
- **Some frequency bands are almost redundant** - if two features are 95% correlated, one of them isn't adding much new information
- **Distant frequencies are less correlated** - low frequencies and high frequencies capture different aspects of the signal

This explains why dimensionality reduction (PCA) might work well here - there's redundancy we can compress.

---

## Part 3: Statistical Study

Time to get serious. Which frequencies actually matter statistically for telling mines from rocks?

In [ ]:
# Statistical tests: T-test and Mann-Whitney U for each frequency band
# T-test assumes normal distribution, Mann-Whitney doesn't

mines_data = df[df['Label'] == 'M'][freq_cols]
rocks_data = df[df['Label'] == 'R'][freq_cols]

stat_results = []

for col in freq_cols:
    mine_vals = mines_data[col]
    rock_vals = rocks_data[col]
    
    # T-test
    t_stat, t_pval = stats.ttest_ind(mine_vals, rock_vals)
    
    # Mann-Whitney U (non-parametric)
    u_stat, u_pval = stats.mannwhitneyu(mine_vals, rock_vals, alternative='two-sided')
    
    # Effect size (Cohen's d)
    pooled_std = np.sqrt((mine_vals.std()**2 + rock_vals.std()**2) / 2)
    cohens_d = (mine_vals.mean() - rock_vals.mean()) / pooled_std if pooled_std > 0 else 0
    
    stat_results.append({
        'Feature': col,
        'Mine_Mean': mine_vals.mean(),
        'Rock_Mean': rock_vals.mean(),
        'Diff': mine_vals.mean() - rock_vals.mean(),
        'T_Statistic': t_stat,
        'T_PValue': t_pval,
        'MW_PValue': u_pval,
        'Cohens_d': cohens_d,
        'Significant': 'Yes' if t_pval < 0.05 else 'No'
    })

stat_df = pd.DataFrame(stat_results)
stat_df = stat_df.sort_values('T_PValue')

print("STATISTICAL SIGNIFICANCE TEST RESULTS")
print("="*50)
print(f"\nSignificant features (p < 0.05): {(stat_df['Significant'] == 'Yes').sum()}/{len(stat_df)}")
print(f"\nTop 10 Most Significant Features:")
print(stat_df[['Feature', 'Mine_Mean', 'Rock_Mean', 'Diff', 'T_PValue', 'Cohens_d', 'Significant']].head(10).to_string(index=False))

**What the stats tell us:**

- **T-test p-value < 0.05** = The difference between mine and rock means is statistically significant (unlikely to be random chance)
- **Cohen's d** = Effect size. How BIG is the difference?
  - |d| < 0.2 = tiny difference
  - |d| ~ 0.5 = medium difference  
  - |d| > 0.8 = large difference

The features at the top of this list are your best discriminators - they show the biggest, most reliable differences between mines and rocks.

---

In [ ]:
# Visualize statistical significance across all frequencies
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# P-values across frequencies
stat_df_sorted = stat_df.sort_values('Feature', key=lambda x: x.str.extract('(\d+)')[0].astype(int))
freq_nums = [int(f.split('_')[1]) for f in stat_df_sorted['Feature']]

colors = ['green' if p < 0.05 else 'red' for p in stat_df_sorted['T_PValue']]
axes[0].bar(freq_nums, -np.log10(stat_df_sorted['T_PValue']), color=colors, edgecolor='black', alpha=0.7)
axes[0].axhline(y=-np.log10(0.05), color='red', linestyle='--', linewidth=2, label='p=0.05 threshold')
axes[0].set_xlabel('Frequency Band')
axes[0].set_ylabel('-log10(p-value)')
axes[0].set_title('Statistical Significance by Frequency Band\n(Higher = more significant, Green = significant)')
axes[0].legend()
axes[0].set_xlim(0, 61)

# Effect size (Cohen's d) across frequencies
colors_d = ['#e74c3c' if d > 0 else '#3498db' for d in stat_df_sorted['Cohens_d']]
axes[1].bar(freq_nums, stat_df_sorted['Cohens_d'], color=colors_d, edgecolor='black', alpha=0.7)
axes[1].axhline(y=0.5, color='green', linestyle='--', alpha=0.7, label='Medium effect (0.5)')
axes[1].axhline(y=-0.5, color='green', linestyle='--', alpha=0.7)
axes[1].axhline(y=0.8, color='orange', linestyle='--', alpha=0.7, label='Large effect (0.8)')
axes[1].axhline(y=-0.8, color='orange', linestyle='--', alpha=0.7)
axes[1].set_xlabel('Frequency Band')
axes[1].set_ylabel("Cohen's d (Effect Size)")
axes[1].set_title("Effect Size by Frequency Band\n(Red = Mines higher, Blue = Rocks higher)")
axes[1].legend(loc='upper right')
axes[1].set_xlim(0, 61)

plt.tight_layout()
plt.show()

**Reading these plots:**

**Top plot (Significance):**
- Bars above the red line = statistically significant differences
- Higher bars = more confident the difference is real
- Middle frequencies tend to be more discriminative

**Bottom plot (Effect Size):**
- Red bars = Mines have higher energy at that frequency
- Blue bars = Rocks have higher energy at that frequency
- Taller bars = bigger practical difference
- Bars beyond the green/orange lines = medium/large effects worth paying attention to

**Pattern emerging:** The signal isn't uniform across frequencies. Some bands are diagnostic goldmines, others are noise.

---

In [ ]:
# Summary statistics
print("STATISTICAL SUMMARY")
print("="*50)
print(f"\nFeatures with p < 0.05:  {(stat_df['T_PValue'] < 0.05).sum()}")
print(f"Features with p < 0.01:  {(stat_df['T_PValue'] < 0.01).sum()}")
print(f"Features with p < 0.001: {(stat_df['T_PValue'] < 0.001).sum()}")
print(f"\nFeatures with |Cohen's d| > 0.5 (medium effect): {(abs(stat_df['Cohens_d']) > 0.5).sum()}")
print(f"Features with |Cohen's d| > 0.8 (large effect):  {(abs(stat_df['Cohens_d']) > 0.8).sum()}")

# Top discriminative features
print(f"\n\nTOP 5 MOST DISCRIMINATIVE FEATURES:")
print("-"*50)
top5 = stat_df.nlargest(5, 'Cohens_d', key=abs)
for _, row in top5.iterrows():
    direction = "Mines > Rocks" if row['Cohens_d'] > 0 else "Rocks > Mines"
    print(f"{row['Feature']:10s} | Cohen's d: {row['Cohens_d']:+.3f} | {direction}")

**The statistical verdict:** A good chunk of frequency bands show significant differences between mines and rocks, but the effect sizes are mostly small to medium. This isn't a slam-dunk separation - it's a subtle pattern recognition problem.

The top discriminative features tell us which frequencies our models should pay most attention to. Spoiler: Random Forest will figure this out automatically.

---

## Part 4: Dimensionality Reduction

60 features is a lot. Can we compress this information while keeping what matters?

In [ ]:
# Prepare data for dimensionality reduction
X = df[freq_cols].values
y = df['Label'].values

# Standardize (important for PCA)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Original shape: {X.shape}")
print(f"Scaled shape: {X_scaled.shape}")
print(f"\nScaled data mean: {X_scaled.mean():.6f} (should be ~0)")
print(f"Scaled data std: {X_scaled.std():.6f} (should be ~1)")

**Why scale?** PCA is sensitive to the scale of features. If one feature ranges from 0-1000 and another from 0-1, PCA would think the first one is more important just because its numbers are bigger. Standardizing puts everyone on equal footing.

---

In [ ]:
# PCA - Principal Component Analysis
pca_full = PCA()
pca_full.fit(X_scaled)

# Explained variance
exp_var = pca_full.explained_variance_ratio_
cum_var = np.cumsum(exp_var)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Individual explained variance
axes[0].bar(range(1, 21), exp_var[:20], color='steelblue', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Principal Component')
axes[0].set_ylabel('Explained Variance Ratio')
axes[0].set_title('Variance Explained by Each Component (Top 20)')
axes[0].set_xticks(range(1, 21))

# Cumulative explained variance
axes[1].plot(range(1, 61), cum_var, 'b-', linewidth=2, marker='o', markersize=4)
axes[1].axhline(y=0.90, color='red', linestyle='--', label='90% variance')
axes[1].axhline(y=0.95, color='orange', linestyle='--', label='95% variance')
axes[1].set_xlabel('Number of Components')
axes[1].set_ylabel('Cumulative Explained Variance')
axes[1].set_title('How Many Components to Keep?')
axes[1].legend()
axes[1].set_xlim(1, 60)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Find components needed for thresholds
n_90 = np.argmax(cum_var >= 0.90) + 1
n_95 = np.argmax(cum_var >= 0.95) + 1

print(f"\nComponents needed for 90% variance: {n_90}")
print(f"Components needed for 95% variance: {n_95}")
print(f"\nFirst 3 components explain: {cum_var[2]*100:.1f}% of variance")
print(f"First 10 components explain: {cum_var[9]*100:.1f}% of variance")

**What PCA revealed:**

- The first few components capture most of the variance (the "important stuff")
- We can go from 60 features down to ~15-20 while keeping 90-95% of the information
- This confirms our earlier observation: there's redundancy in the 60 frequency bands

**The practical implication:** Simpler models might work fine if we reduce dimensions first. Less overfitting risk with fewer features.

---

In [ ]:
# Visualize data in 2D using PCA
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_scaled)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# PCA 2D projection
colors = ['red' if label == 'M' else 'blue' for label in y]
axes[0].scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=colors, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[0].set_xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]*100:.1f}% variance)')
axes[0].set_ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]*100:.1f}% variance)')
axes[0].set_title('PCA: 60D compressed to 2D')

# Add legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='red', edgecolor='black', label='Mine'),
                   Patch(facecolor='blue', edgecolor='black', label='Rock')]
axes[0].legend(handles=legend_elements)

# t-SNE 2D projection (better for visualization, preserves local structure)
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_scaled)

axes[1].scatter(X_tsne[:, 0], X_tsne[:, 1], c=colors, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[1].set_xlabel('t-SNE Dimension 1')
axes[1].set_ylabel('t-SNE Dimension 2')
axes[1].set_title('t-SNE: Nonlinear 2D Projection')
axes[1].legend(handles=legend_elements)

plt.tight_layout()
plt.show()

**Two ways to see 60 dimensions in 2D:**

**PCA (left):** Linear projection. Fast, interpretable. Shows the "main directions" of variation. Notice the classes overlap quite a bit - this problem isn't linearly separable in 2D.

**t-SNE (right):** Non-linear projection. Better at revealing clusters. You might see slightly better separation, but still significant overlap.

**The overlap is the challenge.** There's no magic boundary that cleanly separates all mines from all rocks. This is why this dataset has been a benchmark for decades - it's genuinely hard.

---

In [ ]:
# 3D PCA visualization
from mpl_toolkits.mplot3d import Axes3D

pca_3d = PCA(n_components=3)
X_pca_3d = pca_3d.fit_transform(X_scaled)

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

mines_mask = y == 'M'
rocks_mask = y == 'R'

ax.scatter(X_pca_3d[mines_mask, 0], X_pca_3d[mines_mask, 1], X_pca_3d[mines_mask, 2], 
           c='red', label='Mine', alpha=0.6, s=50)
ax.scatter(X_pca_3d[rocks_mask, 0], X_pca_3d[rocks_mask, 1], X_pca_3d[rocks_mask, 2], 
           c='blue', label='Rock', alpha=0.6, s=50)

ax.set_xlabel(f'PC1 ({pca_3d.explained_variance_ratio_[0]*100:.1f}%)')
ax.set_ylabel(f'PC2 ({pca_3d.explained_variance_ratio_[1]*100:.1f}%)')
ax.set_zlabel(f'PC3 ({pca_3d.explained_variance_ratio_[2]*100:.1f}%)')
ax.set_title(f'3D PCA Projection\n(Total: {sum(pca_3d.explained_variance_ratio_)*100:.1f}% variance explained)')
ax.legend()

plt.tight_layout()
plt.show()

**3D helps a bit:** With three principal components we capture more variance and can see the data from different angles. The classes are still intermingled, but you can start to see regions that are more "mine-heavy" vs "rock-heavy".

**Bottom line on dimensionality reduction:** It's useful for visualization and can help with model performance, but this dataset's fundamental challenge remains - the classes are not easily separable.

---

## Part 5: Model Classification Battle

Alright, this is what we came for. Let's throw some algorithms at this problem and see who survives.

In [ ]:
# Prepare final data
X = df[freq_cols].values
y_encoded = LabelEncoder().fit_transform(df['Label'].values)  # M=0, R=1? Let's check

le = LabelEncoder()
y_encoded = le.fit_transform(df['Label'].values)
print(f"Label encoding: {dict(zip(le.classes_, le.transform(le.classes_)))}")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\nTraining set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Features: {X_train.shape[1]}")

**Setup complete:**
- 80% for training (166 samples), 20% for testing (42 samples)
- Stratified split maintains class proportions in both sets
- Data is scaled (important for SVM, KNN, Neural Networks)

With only 42 test samples, every single prediction matters. One mistake = ~2.4% accuracy drop.

---

In [ ]:
# Define the model army
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'SVM (RBF)': SVC(kernel='rbf', probability=True, random_state=42),
    'SVM (Linear)': SVC(kernel='linear', probability=True, random_state=42),
    'Naive Bayes': GaussianNB(),
    'Neural Network': MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
}

print(f"Models entering the arena: {len(models)}")
for name in models:
    print(f"  - {name}")

**The contenders:**
- **Logistic Regression:** Simple, interpretable, often surprisingly good
- **Decision Tree:** Easy to understand but prone to overfitting
- **Random Forest:** Ensemble of trees, usually robust
- **Gradient Boosting:** Sequential tree building, powerful but slow
- **KNN:** "You are who your neighbors are" - simple but effective
- **SVM (RBF & Linear):** Classic for this dataset, finds optimal separating boundary
- **Naive Bayes:** Fast and simple, assumes feature independence
- **Neural Network:** The deep learning option (not actually deep here, just one hidden layer)

---

In [ ]:
# Train and evaluate all models
results = []
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("Training and evaluating models...\n")
print("-"*85)
print(f"{'Model':<25} | {'Test Acc':>8} | {'CV Mean':>8} | {'CV Std':>7} | {'Precision':>9} | {'Recall':>7}")
print("-"*85)

for name, model in models.items():
    # Use scaled data for distance-based models
    if name in ['K-Nearest Neighbors', 'SVM (RBF)', 'SVM (Linear)', 'Neural Network', 'Logistic Regression']:
        X_tr, X_te = X_train_scaled, X_test_scaled
    else:
        X_tr, X_te = X_train, X_test
    
    # Train
    model.fit(X_tr, y_train)
    
    # Predict
    y_pred = model.predict(X_te)
    
    # Cross-validation on training set
    if name in ['K-Nearest Neighbors', 'SVM (RBF)', 'SVM (Linear)', 'Neural Network', 'Logistic Regression']:
        cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=cv)
    else:
        cv_scores = cross_val_score(model, X_train, y_train, cv=cv)
    
    # Metrics
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    results.append({
        'Model': name,
        'Test_Accuracy': acc,
        'CV_Mean': cv_scores.mean(),
        'CV_Std': cv_scores.std(),
        'Precision': prec,
        'Recall': rec,
        'F1_Score': f1
    })
    
    print(f"{name:<25} | {acc*100:>7.2f}% | {cv_scores.mean()*100:>7.2f}% | {cv_scores.std()*100:>6.2f}% | {prec*100:>8.2f}% | {rec*100:>6.2f}%")

print("-"*85)

**Reading these results:**

- **Test Accuracy:** Performance on the held-out 42 samples
- **CV Mean:** Average accuracy across 5-fold cross-validation (more reliable estimate)
- **CV Std:** How much the accuracy varies across folds (lower = more stable)
- **Precision:** Of all predicted positives, how many were correct?
- **Recall:** Of all actual positives, how many did we catch?

**What to look for:** High test accuracy is good, but CV mean is more trustworthy with small datasets. Big gap between test accuracy and CV mean might indicate lucky/unlucky test split.

---

In [ ]:
# Create results DataFrame and find the champion
results_df = pd.DataFrame(results).sort_values('Test_Accuracy', ascending=False)

print("\n" + "="*60)
print("FINAL LEADERBOARD")
print("="*60)

for rank, (_, row) in enumerate(results_df.iterrows(), 1):
    medal = "[CHAMPION]" if rank == 1 else f"[{rank}]"
    print(f"{medal:12s} {row['Model']:<25} | Test: {row['Test_Accuracy']*100:.2f}% | CV: {row['CV_Mean']*100:.2f}%")

# Champion
champion_name = results_df.iloc[0]['Model']
champion_acc = results_df.iloc[0]['Test_Accuracy']
print(f"\nChampion: {champion_name} with {champion_acc*100:.2f}% test accuracy")

**The moment of truth.** The leaderboard shows who came out on top. Remember, this isn't a 100%-accuracy dataset - anything above 80% is respectable, above 85% is great, and above 90% is excellent.

---

In [ ]:
# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Test accuracy comparison
colors = ['gold' if acc == results_df['Test_Accuracy'].max() else 'steelblue' 
          for acc in results_df['Test_Accuracy']]
bars = axes[0].barh(results_df['Model'], results_df['Test_Accuracy'], color=colors, edgecolor='black')
axes[0].set_xlabel('Accuracy')
axes[0].set_title('Test Set Accuracy')
axes[0].set_xlim(0.5, 1.0)
for bar, acc in zip(bars, results_df['Test_Accuracy']):
    axes[0].text(acc + 0.01, bar.get_y() + bar.get_height()/2, 
                 f'{acc*100:.1f}%', va='center', fontweight='bold')

# CV accuracy with error bars
axes[1].barh(results_df['Model'], results_df['CV_Mean'], 
             xerr=results_df['CV_Std'], color='steelblue', edgecolor='black', capsize=5)
axes[1].set_xlabel('Accuracy')
axes[1].set_title('Cross-Validation Accuracy (with std dev)')
axes[1].set_xlim(0.5, 1.0)

plt.tight_layout()
plt.show()

**Visual comparison:**

- **Left plot:** Raw test accuracy - gold bar is the winner
- **Right plot:** CV accuracy with error bars - shows how stable each model is

Models with small error bars are more reliable - they perform consistently across different data splits. Large error bars mean the model's performance depends heavily on which samples it sees.

---

In [ ]:
# Detailed analysis of the best model
best_model_name = results_df.iloc[0]['Model']
best_model = models[best_model_name]

# Get predictions
if best_model_name in ['K-Nearest Neighbors', 'SVM (RBF)', 'SVM (Linear)', 'Neural Network', 'Logistic Regression']:
    y_pred_best = best_model.predict(X_test_scaled)
    y_prob_best = best_model.predict_proba(X_test_scaled)[:, 1]
else:
    y_pred_best = best_model.predict(X_test)
    y_prob_best = best_model.predict_proba(X_test)[:, 1]

print(f"DETAILED ANALYSIS: {best_model_name}")
print("="*60)
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred_best, target_names=['Mine (M)', 'Rock (R)']))

**Understanding the classification report:**

- **Precision for Mines:** When we predict "Mine", how often are we right? (Important: false alarms waste resources)
- **Recall for Mines:** Of all actual mines, how many did we catch? (CRITICAL: missing a mine = BOOM)
- **F1-Score:** Balance between precision and recall
- **Support:** Number of samples in each class in the test set

In real submarine warfare, you'd probably want to maximize recall for mines (catch every mine, even at the cost of some false alarms).

---

In [ ]:
# Confusion matrix and ROC curve for best model
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_best)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Mine', 'Rock'], yticklabels=['Mine', 'Rock'],
            annot_kws={'size': 16, 'weight': 'bold'})
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')
axes[0].set_title(f'Confusion Matrix: {best_model_name}')

# ROC curve
fpr, tpr, _ = roc_curve(y_test, y_prob_best)
auc = roc_auc_score(y_test, y_prob_best)
axes[1].plot(fpr, tpr, 'b-', linewidth=2, label=f'ROC Curve (AUC = {auc:.3f})')
axes[1].plot([0, 1], [0, 1], 'k--', label='Random Guess')
axes[1].fill_between(fpr, tpr, alpha=0.3)
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title(f'ROC Curve: {best_model_name}')
axes[1].legend(loc='lower right')

plt.tight_layout()
plt.show()

print(f"\nConfusion Matrix Breakdown:")
print(f"  True Mines correctly identified:  {cm[0,0]}")
print(f"  True Rocks correctly identified:  {cm[1,1]}")
print(f"  Mines misclassified as Rocks:     {cm[0,1]} <- DANGEROUS!")
print(f"  Rocks misclassified as Mines:     {cm[1,0]} <- False alarm")

**Reading the confusion matrix:**

- **Top-left (True Mines as Mines):** Good catches
- **Bottom-right (True Rocks as Rocks):** Also good
- **Top-right (Mines as Rocks):** THE DANGEROUS ONES - missed mines
- **Bottom-left (Rocks as Mines):** False alarms - annoying but safe

**ROC Curve:** Shows the trade-off between catching mines (True Positive Rate) and false alarms (False Positive Rate). The closer to the top-left corner, the better. AUC = 1.0 would be perfect.

---

In [ ]:
# Feature importance (if available)
if hasattr(best_model, 'feature_importances_'):
    importance = best_model.feature_importances_
    feat_imp = pd.DataFrame({
        'Feature': freq_cols,
        'Importance': importance
    }).sort_values('Importance', ascending=True)
    
    fig, ax = plt.subplots(figsize=(10, 12))
    colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(feat_imp)))
    ax.barh(feat_imp['Feature'], feat_imp['Importance'], color=colors, edgecolor='black')
    ax.set_xlabel('Importance')
    ax.set_title(f'Feature Importance: {best_model_name}')
    plt.tight_layout()
    plt.show()
    
    print(f"\nTop 10 Most Important Frequencies:")
    for _, row in feat_imp.tail(10).iloc[::-1].iterrows():
        print(f"  {row['Feature']}: {row['Importance']:.4f}")
else:
    print(f"\n{best_model_name} doesn't provide feature importances directly.")
    print("Using Random Forest for feature importance analysis...")
    
    rf = models['Random Forest']
    importance = rf.feature_importances_
    feat_imp = pd.DataFrame({
        'Feature': freq_cols,
        'Importance': importance
    }).sort_values('Importance', ascending=True)
    
    fig, ax = plt.subplots(figsize=(10, 12))
    colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(feat_imp)))
    ax.barh(feat_imp['Feature'], feat_imp['Importance'], color=colors, edgecolor='black')
    ax.set_xlabel('Importance')
    ax.set_title('Feature Importance (Random Forest)')
    plt.tight_layout()
    plt.show()
    
    print(f"\nTop 10 Most Important Frequencies (per Random Forest):")
    for _, row in feat_imp.tail(10).iloc[::-1].iterrows():
        print(f"  {row['Feature']}: {row['Importance']:.4f}")

**Feature importance tells us:** Which frequency bands are actually doing the heavy lifting in classification. The model learned to focus on certain frequencies - these are where the mine/rock signatures differ most.

This connects back to our statistical analysis - the important features here should roughly match the ones with high Cohen's d values.

---

## Part 6: Conclusion

Time to wrap this up. What did we learn?

In [ ]:
print("\n" + "="*70)
print("                      FINAL MISSION REPORT")
print("="*70)

print("""
DATASET SUMMARY:
----------------
- 208 sonar returns (111 mines, 97 rocks)
- 60 frequency band measurements per sample
- Signals from varying viewing angles
- Classic benchmark dataset since 1988

EDA FINDINGS:
-------------
- Classes are fairly balanced (53% mines, 47% rocks)
- No missing values or duplicates
- Mines and rocks have different frequency signatures
- Middle frequency bands show strongest discrimination
- Adjacent frequencies are highly correlated (redundancy exists)

STATISTICAL INSIGHTS:
---------------------""")

sig_features = (stat_df['T_PValue'] < 0.05).sum()
large_effect = (abs(stat_df['Cohens_d']) > 0.5).sum()
print(f"- {sig_features}/60 features statistically significant (p < 0.05)")
print(f"- {large_effect}/60 features with medium+ effect size (|d| > 0.5)")
print(f"- Top discriminator: {stat_df.iloc[0]['Feature']}")

print(f"""
DIMENSIONALITY REDUCTION:
-------------------------
- First {n_90} PCA components capture 90% of variance
- Significant redundancy in the 60 features
- 2D projections show overlapping classes (hard problem!)""")

print(f"""
MODEL PERFORMANCE:
------------------""")
print(f"- Best model: {best_model_name}")
print(f"- Test accuracy: {results_df.iloc[0]['Test_Accuracy']*100:.2f}%")
print(f"- Cross-validation: {results_df.iloc[0]['CV_Mean']*100:.2f}% (+/- {results_df.iloc[0]['CV_Std']*100:.2f}%)")
print(f"- ROC-AUC: {auc:.3f}")

print(f"""
KEY TAKEAWAYS:
--------------
1. This is a genuinely hard classification problem
2. No model achieves 100% - the classes overlap in feature space
3. Ensemble methods and SVMs tend to perform best
4. Middle frequency bands are most informative
5. Cross-validation is essential with small datasets

REAL-WORLD IMPLICATIONS:
------------------------
With {results_df.iloc[0]['Test_Accuracy']*100:.0f}% accuracy, roughly 1 in {int(1/(1-results_df.iloc[0]['Test_Accuracy']))} sonar 
returns would be misclassified. In submarine warfare, that's 
still risky - but way better than random guessing (50%).
""")

print("="*70)
print("                         MISSION COMPLETE")
print("="*70)

---

## The Bottom Line

We took 208 sonar pings, analyzed 60 frequency bands, ran statistical tests, compressed dimensions, and threw 9 different algorithms at the problem.

**What we learned:**
- Mines and rocks DO have different sonar signatures
- But the difference is subtle - not a clean separation
- Machine learning can detect patterns humans would miss
- Even the best models aren't perfect on this data

**Why this matters:**
This isn't just an academic exercise. Sonar classification is used in mine detection, submarine navigation, and underwater exploration. The techniques we used here - statistical analysis, dimensionality reduction, model comparison - are the same ones used in real-world signal processing.

The dataset is small by modern standards, but it's been a benchmark for decades because it captures a fundamental challenge: extracting meaningful patterns from noisy, overlapping signals.

**Next steps if you want to go further:**
- Hyperparameter tuning (GridSearchCV)
- Feature selection (remove redundant frequencies)
- Ensemble stacking (combine multiple models)
- Deep learning (though probably overkill for 208 samples)

---

*Stay safe out there. And if your sonar pings back something weird... maybe don't assume it's just a rock.*

---

**Connect:** [GitHub](https://github.com/Rekhii) | [Kaggle](https://kaggle.com/seki32)